In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
import collections
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
def impute_encoder(data):
    for col in data.columns:
        unique = pd.unique(data[col])
        if len(unique) < 100:
            data[col].fillna(value = data[col].value_counts().index[0], inplace = True)
            fe = data.groupby(col).size() / len(data)
            data.loc[:, col + '_encode'] = data[col].map(fe)
            data.drop([col], axis = 1, inplace = True)
        else:
            data[col].fillna(value = data[col].mean(), inplace = True)
    return data

In [4]:
def ANN(input_shape, leaning_rate):
    
    input_data = tf.keras.Input(shape = input_shape)
    
    x = tf.keras.layers.Dense(32, activation = 'relu', kernel_initializer = 'he_uniform')(input_data)
    
    x = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'he_uniform')(x)
    
    x = tf.keras.layers.Dense(128, activation = 'relu', kernel_initializer = 'he_uniform')(x)
    
    output = tf.keras.layers.Dense(1, activation = 'linear')(x)
    
    model = tf.keras.Model(input_data, output)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'MAE', metrics = ['MeanAbsoluteError'])
    
    return model

In [ ]:
ANN((1, 64), 0.001).summary()

In [5]:
targets = [ 'CIEX', 'CIEY', 'CIEX_DIFF', 'CIEY_DIFF']
data_path = '/home/motionlab/Desktop/weitai/project/smart_manufacturing/csv_file'
train = pd.read_csv(os.path.join(data_path, 'train.csv'))
test = pd.read_csv(os.path.join(data_path, 'test.csv'))
train.drop_duplicates(subset = ['P_ID'], inplace = True)
train_features = train.drop(['id', 'Date', 'LOG ID', 'Sub. ID', 'CIEX', 'CIEY', 'CIEX_DIFF', 'CIEY_DIFF', '4-2_D', '4-2_M'], axis = 1)
test_features = test.drop(['id', 'Date', 'LOG ID', 'Sub. ID', '4-2_D', '4-2_M'], axis = 1)
encode_train_features = impute_encoder(train_features)
encode_test_features = impute_encoder(test_features)
encode_train_features = MinMaxScaler().fit_transform(encode_train_features)
encode_test_features = MinMaxScaler().fit_transform(encode_test_features)
predict_data = np.zeros((5723, 1))
# encode_features
for target in targets:
    model = ANN((None, 1,64), 0.001)
    # 訓練
    model.fit(np.array(encode_train_features), np.array(train[target]), epochs = 30, batch_size = 32, shuffle = True, verbose = 2)
    # 預測
    predict = model.predict(np.array(encode_test_features))
    predict = np.reshape(predict, (5723, 1))
    predict_data = np.concatenate((predict_data, predict), axis = 1)

predict_data = predict_data[:,1:]
predict_dataframe = pd.DataFrame(predict_data, columns = targets)

Epoch 1/30
155/155 - 0s - loss: 0.4848 - mean_absolute_error: 0.4848
Epoch 2/30
155/155 - 0s - loss: 0.1866 - mean_absolute_error: 0.1866
Epoch 3/30
155/155 - 0s - loss: 0.1380 - mean_absolute_error: 0.1380
Epoch 4/30
155/155 - 0s - loss: 0.1194 - mean_absolute_error: 0.1194
Epoch 5/30
155/155 - 0s - loss: 0.0983 - mean_absolute_error: 0.0983
Epoch 6/30
155/155 - 0s - loss: 0.0888 - mean_absolute_error: 0.0888
Epoch 7/30
155/155 - 0s - loss: 0.0773 - mean_absolute_error: 0.0773
Epoch 8/30
155/155 - 0s - loss: 0.0780 - mean_absolute_error: 0.0780
Epoch 9/30
155/155 - 0s - loss: 0.0771 - mean_absolute_error: 0.0771
Epoch 10/30
155/155 - 0s - loss: 0.0675 - mean_absolute_error: 0.0675
Epoch 11/30
155/155 - 0s - loss: 0.0608 - mean_absolute_error: 0.0608
Epoch 12/30
155/155 - 0s - loss: 0.0664 - mean_absolute_error: 0.0664
Epoch 13/30
155/155 - 0s - loss: 0.0635 - mean_absolute_error: 0.0635
Epoch 14/30
155/155 - 0s - loss: 0.0537 - mean_absolute_error: 0.0537
Epoch 15/30
155/155 - 0s - lo

In [ ]:
predict_dataframe

In [6]:
submmit = pd.concat([test['id'], predict_dataframe], axis = 1)
submmit

,id,CIEX,CIEY,CIEX_DIFF,CIEY_DIFF
0,2,2.892117,2.120482,0.006253,0.009253
1,4,2.918307,2.117813,0.005398,0.010300
2,5,2.890807,2.117615,0.006278,0.009298
3,7,2.892739,2.115937,0.006440,0.009275
4,10,2.913378,2.122495,0.005771,0.009868
...,...,...,...,...,...
5718,10674,0.761372,4.815085,0.005986,0.008908
5719,10675,0.698064,4.829456,0.016087,0.002427
5720,10676,0.752968,4.809276,0.005568,0.008629
5721,10680,0.692284,4.821939,0.016178,0.002561


In [7]:
submmit.to_csv('/home/motionlab/Desktop/weitai/project/test result/ann_first_trial.csv', index = False)